## Production Amounts and Scope 1 and 2 Emission Values Per EAF Plant in the United States

### Loading Libraries

In [43]:
import pandas as pd
import numpy as np
import janitor
import geopandas as gpd
import mapclassify as mc
from shapely.geometry import Point

### Importing eGrid Data

The EPA eGrida Data, which was available for 2021 at the time of download, contains the emissions intensity of electricity produced in each sub region. For this analysis, we assume that each steel plant is drawing 100% of its electricity from the grid (and the eGrid subregion that it is located within).

In [44]:
## Importing grid region electricity values
egrid2021_data = pd.read_excel('../data/eGRID2021_data.xlsx', sheet_name = "SRL21").clean_names(strip_underscores = True).drop(0)

### Standardizing state names and cleaning up rows in eGrid data

In [45]:
## Cleaning egrid data for usability
emissions_by_subregion = egrid2021_data.copy()

emissions_by_subregion["co2e_lbs_per_mwh"] = pd.to_numeric(emissions_by_subregion["egrid_subregion_annual_co2_equivalent_combustion_output_emission_rate_lb_mwh"])
emissions_by_subregion['subregion'] = emissions_by_subregion['egrid_subregion_acronym']
emissions_by_subregion['subregion_name'] = emissions_by_subregion['egrid_subregion_name']
emissions_by_subregion['co2e_tonnes_per_mwh'] = emissions_by_subregion['co2e_lbs_per_mwh'] / 2204.62262185

# Filtering subregions
excluded_subregions = ["AKGD", "AKMS", "HIMS", "HIOA", "PRMS"]
emissions_by_subregion = emissions_by_subregion[~emissions_by_subregion['subregion'].isin(excluded_subregions)]

# Selecting desired columns
emissions_by_subregion = emissions_by_subregion[['subregion', 'subregion_name', 'co2e_tonnes_per_mwh']]

### Importing GEM data

In [46]:
gem_data_readin = pd.read_excel("../data/GEM_2022_data.xlsx", sheet_name = "Steel Plants").clean_names(strip_underscores = True)

### Filtering to only look at plants and data that we are interested in

In [47]:
## eaf_capacity is in thousand tonnes per year and we are standardizing to tonnes
gem_data_cleaned = gem_data_readin.copy()

## Filtering to the specifications we need
gem_data_cleaned = gem_data_cleaned[
       (gem_data_cleaned['country'] == "United States") &
       (gem_data_cleaned["status"] == "operating") &
       gem_data_cleaned['nominal_eaf_steel_capacity_ttpa'].notna()
]

## Needed to do this in another step to make sure start_date was properly filtered
gem_data_cleaned = gem_data_cleaned[
       (gem_data_cleaned['start_date'] < 2022) &
       (~gem_data_cleaned['plant_id'].isin(["SUS00009", "SUS00061"]))
]

gem_data_cleaned = gem_data_cleaned.rename(columns={'plant_name_english':'plant_name'
                                          , 'subnational_unit_province_state':'state'
                                          , 'location_address':'address'})

gem_data_cleaned['eaf_capacity'] = pd.to_numeric(gem_data_cleaned['nominal_eaf_steel_capacity_ttpa'])
gem_data_cleaned['max_tonnes_of_steel_producible_annually'] = gem_data_cleaned['eaf_capacity'] * 1000

gem_data_cleaned = gem_data_cleaned[['plant_id'
         , 'plant_name'
         , 'owner'
         , 'coordinates'
         , 'country'
         , 'state'
         , 'status'
         , 'start_date'
         , 'plant_age_years'
         , 'max_tonnes_of_steel_producible_annually'
         , 'municipality'
         , 'address'
         , 'category_steel_product'
         , 'steel_products'
         , 'responsiblesteel_certification']]

## Removing columns we do not need
gem_data = gem_data_cleaned.drop(columns=['country', 'start_date', 'status', 'responsiblesteel_certification'])

## Separate the "coordinates" column into "lat" and "lon" columns
gem_data[['lat', 'lon']] = gem_data['coordinates'].str.split(',', expand=True)

## Remove the "coordinates" column
gem_data.drop(columns=['coordinates'], inplace=True)

## Reordering columns
gem_data = gem_data[['plant_id', 'plant_name', 'owner', 'lat', 'lon', 'state', 'plant_age_years', 'max_tonnes_of_steel_producible_annually', 'municipality', 'address', 'category_steel_product', 'steel_products']]

In [48]:
## Reading in data
subregion_shapes_raw = gpd.read_file("../data/egrid2020_subregions/eGRID2020_subregions.shp").clean_names()

## Filtering subregion shapes
subregion_shapes = subregion_shapes_raw[~subregion_shapes_raw['zipsubregi'].isin(["AKGD", "AKMS", "HIMS", "HIOA", "PRMS"])]

## Simplifying subregion shapes
subregion_shapes['geometry'] = subregion_shapes.simplify(tolerance=0.0005)

## Bringing in plant points
plant_points = gpd.GeoDataFrame(gem_data, geometry=gpd.points_from_xy(gem_data['lon'], gem_data['lat']))
plant_points.crs = "EPSG:4326"

## Finding points that overlap with the egrid subregion
## Removing columns we dont need and renaming the subregion column
plant_emissions_by_subregion = gpd.sjoin(plant_points, subregion_shapes, op='within').drop(columns=['geometry', 'index_right', 'shape_leng', 'shape_le_1', 'shape_area']).rename(columns={'zipsubregi':'subregion'})

plant_emissions_by_subregion = plant_emissions_by_subregion.merge(emissions_by_subregion, on='subregion', how='left')

/Users/mikey/opt/miniconda3/envs/mikeys_env/lib/python3.10/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


### Importing AISI data

In [49]:
AISI_regions_readin = pd.read_excel("../data/AISI_regions.xlsx", sheet_name="Regions by State").clean_names(strip_underscores = True)

AISI_data_readin = pd.read_excel("../data/AISI_data.xlsx", sheet_name="AISI Production Values").clean_names(strip_underscores = True)

NE = Northeast

GL = Great Lakes

MW = Midwest

S = Southern

W = Western

### Filtering AISI data, renaming columns, and selecting the weekly data and utilization rates we need

**Uilization is based on tonnage capability to produce raw steel for a sustained full order book.**

In [67]:
AISI_regions = AISI_regions_readin[AISI_regions_readin['steel_plant_in_gspt'] == "yes"]

AISI_data = AISI_data_readin.rename(columns={'north_east_region_capacity_utilization':'NE_util'
                                          , 'great_lakes_region_capacity_utilization':'GL_util'
                                          , 'midwest_region_capacity_utilization':'MW_util'
                                          , 'southern_region_capacity_utilization':'S_util'
                                          , 'western_region_capacity_utilization':'W_util'})

AISI_data = AISI_data[['week_end_date', 'NE_util', 'GL_util', 'MW_util', 'S_util', 'W_util']]

AISI_data = AISI_data[AISI_data['week_end_date'] <= '2022-01-01']

### Merging all of our data so far

In [68]:
regional_plant_emissions = merge()

"""""
regional_plant_emissions <- left_join(plant_emissions_by_subregion, AISI_regions, by = "state") %>% 
  select(-c(steel_plant_in_gspt
            , state_abbreviation)) %>% 
  mutate(region = case_when(municipality %in% c("Alton"
                                                , "Sterling"
                                                , "Peoria"
                                                , "Granite City"
                                                , "Mansfield"
                                                , "Middletown") ~ "Midwest"
                            , municipality %in% c("Riverdale"
                                                  , "Chicago"
                                                  , "Bourbonnais"
                                                  , "Cuyahoga Heights"
                                                  , "Cleveland"
                                                  , "Toledo"
                                                  , "Lorain") ~ "Great Lakes"
                            , municipality %in% c("Mingo Junction"
                                                  , "Youngstown"
                                                  , "Canton") ~ "North East"
                            , TRUE ~ as.character(region)))
                            """

,week_end_date,NE_util,GL_util,MW_util,S_util,W_util
0,2021-01-09,0.698035,0.706488,0.584808,0.863905,0.802078
1,2021-01-16,0.711545,0.729999,0.610235,0.858961,0.824995
2,2021-01-23,0.689028,0.721770,0.600700,0.855254,0.790620
3,2021-01-30,0.671014,0.714717,0.591165,0.877500,0.824995
4,2021-02-06,0.662007,0.704137,0.578452,0.875028,0.790620
5,2021-02-13,0.698035,0.704137,0.600700,0.896039,0.859370
6,2021-02-20,0.689028,0.748807,0.581630,0.865141,0.824995
7,2021-02-27,0.698035,0.733525,0.575274,0.883680,0.847911
8,2021-03-06,0.707042,0.715892,0.568917,0.914578,0.802078
9,2021-03-13,0.693532,0.737052,0.578452,0.908398,0.721870
